In [ ]:
play_count_csv_path = "..\\MPC\\play_count.csv"

In [ ]:
import time


In [ ]:
with open("..\\playCount.txt", encoding='utf-8') as f:
    lines = f.readlines()

In [ ]:
import subprocess

def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [ ]:
def get_size(file_name):
    file_stats = os.stat(file_name)
    file_stats = file_stats.st_size / (1024 * 1024)
    
    return file_stats

In [ ]:
current_time = time.time()

In [ ]:


def days_since_file_creation(path):
    if os.path.exists(path):
        # Get the time the file was created/added to the disk
        file_creation_time = os.path.getctime(path)
        
        # Get the current time
        
        
        # Calculate the time difference in seconds
        time_diff_seconds = current_time - file_creation_time
        
        # Convert time difference to days
        time_diff_days = time_diff_seconds / (60 * 60 * 24)
        
        return time_diff_days
    
    else:
        pass
        #print(f"{path} does not exist.")

In [ ]:
print("hello")


In [ ]:

import os
from operator import itemgetter
import win32con
import win32gui
import pyautogui
import time
#import pywintypes
#from IPython.terminal import console
#import jupyter_console
from win32con import (SW_SHOW, SW_RESTORE)
from send2trash import send2trash
global playIndex
import random as rand
from pynput import keyboard
from tqdm import tqdm

from pynput.mouse import Listener as mouseListener, Button

from pynput.keyboard import Key
import pickle

import vlc

In [ ]:
import KillDrivers

In [ ]:
autoplaying = False

uniqueOnly = True

recentListLive = []

In [ ]:


directoryList = ['..\\35']

play_count_path ="..\\playCount.txt"

play_count_csv_path = "..\\play_count.csv"

recent_played_path = '..\\recentplayed.txt'

dup_list_path = "..\\33\\dupList.txt"


sleepTime = 0.3

def readAlreadyKeep():
    newlist = []

    with open("..\\keep.txt", 'r', encoding='cp1252') as f:
        contents = f.readlines()
            
   
        return contents



def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory
    counter = 0
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        try:
            fullPath = os.path.join(dirName, entry)
            # If entry is a directory then get the list oFf files in this directory
            if os.path.isdir(fullPath):
                allFiles = allFiles + getListOfFiles(fullPath)
            else:
                allFiles.append(fullPath)
        except:
            counter = counter + 1
            pass
    

    keptFiles = readAlreadyKeep()
    

    if(counter > 5):
        print("Failed on: ", counter )

    return allFiles

def buildFileNameFileSize(directory):    
    ls=  getListOfFiles(directory)
    
    nameSizeList = []
    
    for i in range(0, len(ls)):
        try:

            nameSizeList.append((ls[i], os.stat(ls[i]).st_size/1000, os.stat(ls[i]).st_atime))
           
        except:
            pass
    
    return nameSizeList
       

def getSortedList(OneForSizeTwoForAccessed, directory):
    global OSTA
    OSTA = OneForSizeTwoForAccessed
    a = buildFileNameFileSize(directory) #True == sort by last access, false for size
    sortedlist = sorted(a, key=itemgetter(OneForSizeTwoForAccessed))
    if(OSTA == 1):
        sortedlist.reverse()
    #print(sortedlist[0])
    print("Sorted")
    print(len(sortedlist))
    
    
    ##randomise by half
#     firstHalf = sortedlist[0:int(len(sortedlist)/2)]
#     secondHalf = sortedlist[int(len(sortedlist)/2):len(sortedlist)]
    
#     random.shuffle(firstHalf)
    
#     sortedlist = firstHalf+secondHalf

    
    return sortedlist

def cleanupSortedList(slist):
    newList = []
    
    for i in range(0, len(slist)):
        if(slist[i][1]>25000 and not slist[i][0].endswith('Zip"') 
           and not slist[i][0].lower().endswith('zip"')
           and not slist[i][0].lower().endswith('rar"')
           and not slist[i][0].lower().endswith('pdf"') 
           and not slist[i][0].lower().endswith('mpcpl"')
           and not slist[i][0].lower().endswith('dat"')
           and not slist[i][0].lower().endswith('vob"')
           and not slist[i][0].lower().endswith('7zip"')
           and len(slist[i][0])>0  ): #and '7z' not in slist[i][0]
         # and 'pdf' not in slist[i][0] and 'zip' not in slist[i][0]):
            #print(slist[i][0])
            newList.append(slist[i])
    print("Cleaned up:")        
    print(len(newList))        
    return newList

def filterList(slist, searchTerm):
    newList = []


    for i in range(0, len(slist)):
        if(searchTerm in slist[i][0].lower()
        and slist[i][1]>25000
        and not slist[i][0].endswith('Zip"')
           and not slist[i][0].lower().endswith('zip"')
           and not slist[i][0].lower().endswith('rar"')
           and not slist[i][0].lower().endswith('pdf"')
           and not slist[i][0].lower().endswith('mpcpl"')
           and not slist[i][0].lower().endswith('dat"')
           and not slist[i][0].lower().endswith('vob"')
           and not slist[i][0].lower().endswith('7zip"')
           and not slist[i][0].lower().endswith('jpg"')
           and not slist[i][0].lower().endswith('png"')
           and not slist[i][0].lower().endswith('jpeg"')
           and len(slist[i][0])>0 ):
            newList.append(slist[i])
    print("filtered:")
    print(len(newList))
    for i in range(0, len(newList)):
        print(newList[i][0].lower())
    return newList


def decVol():
    player.audio_set_volume(abs(player.audio_get_volume()-10))
    print(player.audio_get_volume())

def incVol():
    player.audio_set_volume(abs(player.audio_get_volume()+10))
    print(player.audio_get_volume())

def toggleFullscreen():
    if(player.get_fullscreen() == 0):
        player.set_fullscreen(True)
    else:
        player.set_fullscreen(False)

def getIncIntTiny(pos):
    
    return pos + (pos / 15)/10

def getDecIntTiny(pos):
    
    return pos - (pos / 15)/10

def getDeccIntMed(pos):
   
    return pos - (pos / 3)/10

def getIncIntMed(pos):
    
    return pos + (pos / 3)/10

def getDeccIntLrg(pos):
    
    return pos - 0.04

def getIncIntLrg(pos):
    
    return pos + 0.06


def skipFwdTiny():
    player.set_position(getIncIntTiny(player.get_position()))

def skipBwdTiny():
    player.set_position(getDecIntTiny(player.get_position()))

def skipFwdMed():
    player.set_position(getIncIntMed(player.get_position()))

def skipBwdMed():
    player.set_position(getDeccIntMed(player.get_position()))
    
    
def skipFwdLrg(player):
    
    
    player.set_position(getIncIntLrg(player.get_position()))
    if(player.get_position() > 0.99):
        print("Replaying end")
        #player = playAgain(player)
        player.set_position(0.1)
        return player
    return player

def skipBwdLrg():
    player.set_position(getDeccIntLrg(player.get_position()))    
    
    return player
    
    
    
def pause():
    player.pause()
    

def window_enum_handler(hwnd, resultList):
    if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd) != '':
        resultList.append((hwnd, win32gui.GetWindowText(hwnd)))

def get_app_list(handles=[]):
    mlst=[]
    win32gui.EnumWindows(window_enum_handler, handles)
    for handle in handles:
        mlst.append(handle)
    return mlst

def get_windows_placement(window_id):
    return win32gui.GetWindowPlacement(window_id)[1]

def set_active_window(window_id):
    if get_windows_placement(window_id) == 2:
        win32gui.ShowWindow(window_id, SW_RESTORE)
    else:
        win32gui.ShowWindow(window_id, SW_SHOW)
    #win32gui.SetForegroundWindow(window_id)
    win32gui.SetActiveWindow(window_id)

def addPlaylist(player):
        mediaList = player.media_list_new()
        path = r""
        songs = os.listdir(path)
        for s in songs:
            mediaList.add_media(Player.media_new(os.path.join(path,s)))
        listPlayer = player.media_list_player_new()
        listPlayer.set_media_list(mediaList)
        return listPlayer

def playNext():
    media_player.next()
    time.sleep(3)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()

def isVLCOpen():
    appwindows = get_app_list()
    for i in appwindows:
    
        if('VLC (Direct3D11 output)' in i[1]):
            
            return True
    
    return False

def playAgain(player):
    global playIndex
    
    
    #player.stop()
    
    playIndex
    
    media = vlc.Media(cPlaylist[playIndex])
    player.set_media(p_md=media)
    
    #player = vlc.MediaPlayer(cPlaylist[playIndex])
    player.play()
    
    time.sleep(sleepTime)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()
    if(isVLCOpen() is False):
        print("Playback failed on " + str(cPlaylist[playIndex]))
        player = playNextRand()
    
    return player

def playNextInc(player):
    global playIndex
    
    
    #player.stop()
    
    playIndex = playIndex + 1
    
    #player = vlc.MediaPlayer(cPlaylist[playIndex][0])
    #player.set_media(cPlaylist[playIndex][0])
    media = vlc.Media(cPlaylist[playIndex][0])
    player.set_media(p_md=media)
    player.play()
    
    time.sleep(sleepTime)
    if(player.get_fullscreen() == 0):
        player.set_fullscreen(True)
    else:
        win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()
    if(isVLCOpen() is False):
        print("Playback failed on " + str(cPlaylist[playIndex][0]))
        playNextRand()
    
    return player



def playPrev():
    media_player.previous()
    time.sleep(sleepTime)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()

def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))

def bringWindowToForeground():
    global wasFullScreen
    print("Bringing window")
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for window in top_windows:
        if("VLC (Direct3D11 output)" in window[1]):
            try:
                #set_active_window(pywintypes.HANDLE(window[0]))
                #win32gui.ShowWindow(window[0])
                #win32gui.SetForegroundWindow(window[0])
                #win32gui.set_focus(window[0])
                raise_window("VLC (Direct3D11 output)")

                if(wasFullScreen):
                    w=win32gui
                    res = w.GetWindowText (w.GetForegroundWindow())
                    if('vlc' in  res.lower()):
                        print('bringing window foreground maximising vlc in title')
                        hwnd = win32gui.GetForegroundWindow()
                        print('hwnd', hwnd)
                        win32gui.ShowWindow(hwnd, win32con.SW_MAXIMIZE)

            except Exception as e:
                print("Failed to bring forward")
                print(e)
                time.sleep(5)
                print("recursive call")
                bringWindowToForeground()
                pass






def resizeVLCRight(hwnd, lParam):
    if win32gui.IsWindowVisible(hwnd):
        if 'VLC' in win32gui.GetWindowText(hwnd):
            win32gui.MoveWindow(hwnd, 1940, 0, 1900, 2110, True)

#win32gui.EnumWindows(resizeVLCRight, None)

def addFilesToPlaylist(cPlaylist):
    
    player = vlc.Instance()
    media_player = vlc.MediaListPlayer()
    media_list = player.media_list_new()
    for file in cPlaylist:
        media = player.media_new(file[0])
        media_list.add_media(media)
        
    media_player.set_media_list(media_list)
    return player, media_player

def sendIndexToRecyclebin(playIndex):
    global player
    global cPlaylist
    toDelete = cPlaylist[playIndex]
    player = playNextRand()
    #player.stop()
    time.sleep(1)

    print("deleting...to delete" + cPlaylist[playIndex], toDelete)

    #player = playNextRand()
    send2trash(toDelete)
    #player = playNextInc(player)
    #player = playNextRand()

    return player




In [ ]:
def decVol():
    player.audio_set_volume(abs(player.audio_get_volume()-10))
    print(player.audio_get_volume())

def incVol():
    player.audio_set_volume(abs(player.audio_get_volume()+10))
    print(player.audio_get_volume())

def toggleFullscreen():
    if(player.get_fullscreen() == 0):
        player.set_fullscreen(True)
        try:
            if(not pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive or pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isMinimized or not pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].visible):
                print("Maximising window")
                pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].maximize()
                player.set_fullscreen(True)
        except Exception as e:
            print("Failed to maximise window or checks")
            print(e)


    else:
        player.set_fullscreen(False)

def getIncIntTiny(pos):
    
    return pos + (pos / 15)/10

def getDecIntTiny(pos):
    
    return pos - (pos / 15)/10

def getDeccIntMed(pos):
   
    return pos - (pos / 3)/10

def getIncIntMed(pos):
    
    return pos + (pos / 3)/10

def getDeccIntLrg(pos):
    
    return pos - 0.04

def getIncIntLrg(pos):
    
    return pos + 0.06


def skipFwdTiny():
    player.set_position(getIncIntTiny(player.get_position()))

def skipBwdTiny():
    player.set_position(getDecIntTiny(player.get_position()))

def skipFwdMed():
    player.set_position(getIncIntMed(player.get_position()))

def skipBwdMed():
    player.set_position(getDeccIntMed(player.get_position()))
    
    
def skipFwdLrg(player):
    
    global wasFullScreen

    player.set_position(getIncIntLrg(player.get_position()))


    # if(wasFullScreen):
    #             print('bringing window foreground maximising')
    #             hwnd = win32gui.GetForegroundWindow()
    #             win32gui.ShowWindow(hwnd, win32con.SW_MAXIMIZE)

    if(player.get_position() > 0.99):
        print("Replaying end")
        #player = playAgain(player)
        player.set_position(0.1)
        return player
    return player

def skipBwdLrg():
    player.set_position(getDeccIntLrg(player.get_position()))    
    if(player.get_position() < 0.1):
        player.set_position(0.99)
    
    
    return player
    
    
    
def pause():
    player.pause()


In [ ]:
def raise_window(my_window):

    import win32con
    import win32gui

    def get_window_handle(partial_window_name):

        # https://www.blog.pythonlibrary.org/2014/10/20/pywin32-how-to-bring-a-window-to-front/

        def window_enumeration_handler(hwnd, windows):
            windows.append((hwnd, win32gui.GetWindowText(hwnd)))

        windows = []
        win32gui.EnumWindows(window_enumeration_handler, windows)

        for i in windows:
            if partial_window_name.lower() in i[1].lower():
                return i
                break

        print('window not found!')
        return None

    # https://stackoverflow.com/questions/6312627/windows-7-how-to-bring-a-window-to-the-front-no-matter-what-other-window-has-fo

    def bring_window_to_foreground(HWND):
        win32gui.ShowWindow(HWND, win32con.SW_RESTORE)
        win32gui.SetWindowPos(HWND, win32con.HWND_NOTOPMOST, 0, 0, 0, 0, win32con.SWP_NOMOVE + win32con.SWP_NOSIZE)
        win32gui.SetWindowPos(HWND, win32con.HWND_TOPMOST, 0, 0, 0, 0, win32con.SWP_NOMOVE + win32con.SWP_NOSIZE)
        win32gui.SetWindowPos(HWND, win32con.HWND_NOTOPMOST, 0, 0, 0, 0, win32con.SWP_SHOWWINDOW + win32con.SWP_NOMOVE + win32con.SWP_NOSIZE)

    hwnd = get_window_handle(my_window)

    if hwnd is not None:
        bring_window_to_foreground(hwnd[0])

In [ ]:
def window_enum_handler(hwnd, resultList):
    if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd) != '':
        resultList.append((hwnd, win32gui.GetWindowText(hwnd)))

def get_app_list(handles=[]):
    mlst=[]
    win32gui.EnumWindows(window_enum_handler, handles)
    for handle in handles:
        mlst.append(handle)
    return mlst

In [ ]:
def get_windows_placement(window_id):
    return win32gui.GetWindowPlacement(window_id)[1]

def set_active_window(window_id):
    if get_windows_placement(window_id) == 2:
        win32gui.ShowWindow(window_id, SW_RESTORE)
    else:
        win32gui.ShowWindow(window_id, SW_SHOW)
    win32gui.SetForegroundWindow(window_id)
    win32gui.SetActiveWindow(window_id)

In [ ]:
def addPlaylist(player):
        mediaList = player.media_list_new()
        path = r""
        songs = os.listdir(path)
        for s in songs:
            mediaList.add_media(Player.media_new(os.path.join(path,s)))
        listPlayer = player.media_list_player_new()
        listPlayer.set_media_list(mediaList)
        return listPlayer

In [ ]:
def playNext():
    media_player.next()
    time.sleep(3)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()

In [ ]:
def isVLCOpen():
    appwindows = get_app_list()
    for i in appwindows:
    
        if('VLC (Direct3D11 output)' in i[1]):
            
            return True
    
    return False

In [ ]:
def isTwoVLC():
    appwindows = get_app_list()
    
    return len(appwindows)>0

In [ ]:
def playAgain():
    global playIndex
    global player
    
    #player.stop()
    
    #playIndex
    
    #player = vlc.MediaPlayer(cPlaylist[playIndex])
    media = vlc.Media(cPlaylist[playIndex])
    player.set_media(p_md=media)
    
    player.play()
    
    time.sleep(sleepTime)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()
    if(isVLCOpen() is False):
        print("Playback failed on " + str(cPlaylist[playIndex]))
        player = playNextRand()
    
    return player

In [ ]:
def playNextInc(player):
    global playIndex
    
    
    #player.stop()
    
    playIndex = playIndex + 1
    
    #player = vlc.MediaPlayer(cPlaylist[playIndex])
    media = vlc.Media(cPlaylist[playIndex])
    player.set_media(p_md=media)
    player.play()
    
    time.sleep(2)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()
    if(isVLCOpen() is False):
        print("Playback failed on " + str(cPlaylist[playIndex]))
        playNextRand()
    
    return player

In [ ]:
enableResize = True

def playNextRand():
    global playIndex
    global player
    global uniqueOnly
    global recentListLive
    global rec2
    global wasFullScreen
    global currentlyPlaying
    global cPlaylist

    #player.stop()
    
    playIndex = rand.randint(0, len(cPlaylist))
    
    recentList = readRecentPlays()

    global currentlyPlaying
    global autoplaying

    wasFullScreen = player.get_fullscreen() == 1

    b_file = open(play_count_path, "r", encoding="utf-8")

    linesStripped = []

    lines = b_file.readlines()

    for line in lines:

        #linesStripped.append(' '.join(line.split()[0:len(line.split())-1]))
        try:
            linesStripped.append((line[0:line.rfind((line.split()[-1]))-1], int(line.split()[-1])))
        except Exception as e:
            print(e)
            print(line)



    b_file.close()
    foundNew = False
    for fileName in cPlaylist:
        if fileName not in linesStripped and foundNew is False:
            print("Found new file ", fileName)
            print(linesStripped[0])
            foundNew = True
            if(autoplaying == False ):
                writeLine(fileName)
            try:
                print("loading new fileName into player")
                #player = vlc.MediaPlayer(fileName)
                
                media = vlc.Media(fileName)
                player.set_media(media)
                player.play()
                cPlaylist.remove(fileName)
                if(autoplaying == False ):
                    updatePlayCount(fileName)
                currentlyPlaying = fileName

                time.sleep(sleepTime*4)
                if(enableResize):
                    win32gui.EnumWindows(resizeVLCRight, None)
                    bringWindowToForeground()
                return player
            except Exception as e:
                print(e)
                print("Failed in play next rand 2 trying recursive")
                break
                playNextRand()




    if(foundNew is False):
        if(uniqueOnly):
            if(rand.randint(0, 1)>0.6):
                print("selecting random index")
                playIndex = rand.choice(cPlaylist)
            else:
                while cPlaylist[playIndex] in recentList:
                    print(cPlaylist[playIndex] + " in recent list")

                    print("Getting new playIndex")


                    playIndex = rand.randint(0, len(cPlaylist))
        else:
            print("Getting new playIndex")
            if(len(recentListLive) <1):
                print("Resetting recent list live")
                recentListLive = []

            while cPlaylist[playIndex] in recentListLive:
                print("Getting new playIndex")
                playIndex = rand.randint(0, len(cPlaylist))

        a_file = open(play_count_path, "r", encoding="utf-8")

        lines1 = a_file.readlines()

        a_file.close()

        strippedLines = []
        for line1 in lines1:

            strippedLines.append((line1[0:line1.rfind((line1.split()[-1]))-1], int(line1.split()[-1])))

        if(len(strippedLines) == len(cPlaylist)):
            print("Selecting unique, cPlaylist == strippedlines")
            # if(cPlaylist[playIndex] in strippedLines):
            tupleList = []

            for line2 in lines1:

                tupleList.append((line2[0:line2.rfind((line2.split()[-1]))-1], int(line2.split()[-1])))



            freshPlay = sorted(tupleList,key=itemgetter(1))[0][0]
            cPlaylist[playIndex] = freshPlay
            print("Next to play ", freshPlay)




        # while cPlaylist[playIndex] in recentList:
        #     print(cPlaylist[playIndex] + " in recent list")
        #
        #     print("Getting new playIndex")


            #playIndex = rand.randint(0, len(cPlaylist))


        #if(cPlaylist[playIndex][0] not in recentList):
        if(autoplaying == False ):
            updatePlayCount(cPlaylist[playIndex])

        if(not any(cPlaylist[playIndex] in string for string in recentList) or uniqueOnly or not any(cPlaylist[playIndex] in string for string in rec2)):
            print("Writing " + str(cPlaylist[playIndex]))
            if(uniqueOnly):
                if(autoplaying == False ):
                    writeLine(cPlaylist[playIndex])

                playCounttext = readPlayCount()

                #writeLinePlayCount((cPlaylist[playIndex], 1))
                #updatePlayCount(cPlaylist[playIndex])
                # if(cPlaylist[playIndex] not in playCounttext):
                #     writeLinePlayCount((cPlaylist[playIndex], 1))
                #     updatePlayCount(cPlaylist[playIndex])
                #
                # else:
                #     print("updating playcount")
                #     updatePlayCount(cPlaylist[playIndex])


            else:
                recentListLive.append(cPlaylist[playIndex])
            print(cPlaylist[playIndex])

            # if(isTwoVLC()):
            #     print("Closing double window")
            #     player.stop()
            #
            #     appwindows = get_app_list()
            #
            #     try:
            #         pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].close()
            #
            #     except Exception as e:
            #         print(e)
            #
            #     try:
            #         pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[1].close()
            #
            #     except Exception as e:
            #         print(e)
            try:
                print("TRying to play ", playIndex)
                #player = vlc.MediaPlayer(cPlaylist[playIndex])
                
                #player.set_media(cPlaylist[playIndex])
                media = vlc.Media(cPlaylist[playIndex])
                player.set_media(p_md=media)
                player.play()

                time.sleep(sleepTime*4)
                if(enableResize):
                    win32gui.EnumWindows(resizeVLCRight, None)
                    bringWindowToForeground()
            except Exception as e:
                print(e)
                print("Failed in play next rand 2 trying recursive")
                playNextRand()

            if(isVLCOpen() is False):
                print("Playback failed on " + str(cPlaylist[playIndex]))
                #playNextRand()

            time.sleep(sleepTime*4)
            if(enableResize):
                win32gui.EnumWindows(resizeVLCRight, None)
                bringWindowToForeground()

        else:
            print("Recursive call")
            print(cPlaylist[playIndex])
            playIndex = rand.randint(0, len(cPlaylist))

            playNextRand()

        #time.sleep(1)
        try:
            if(not pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive or pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isMinimized or not pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].visible):
                print("Maximising window")
                pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].maximize()
        except Exception as e:
            print("Failed to maximise window or checks")
            print(e)
            print("Making recursive call after failed window")
            playNextRand()

        bringWindowToForeground()

        if(wasFullScreen):
            print("was full screen setting2")
            #player.set_fullscreen(True)
            if(player.get_fullscreen() == 0):
                toggleFullscreen()
        else:
            print("was not full screen setting2")
            #player.set_fullscreen(False)
            if(player.get_fullscreen() == 1):
                toggleFullscreen()


    return player

In [ ]:
#cPlaylist[playIndex][0]

In [ ]:
def playPrev():
    media_player.previous()
    time.sleep(sleepTime)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()

In [ ]:
def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))

In [ ]:


def resizeVLCRight(hwnd, lParam):
    if win32gui.IsWindowVisible(hwnd):
        if 'VLC (Direct3D11 output)' in win32gui.GetWindowText(hwnd):
            win32gui.MoveWindow(hwnd, 1940, 0, 1900, 2110, True)

#win32gui.EnumWindows(resizeVLCRight, None)

In [ ]:
def writeLine(line):
    with open(recent_played_path, encoding="utf-8") as fobj:
        text = fobj.read()
    
    with open(recent_played_path, "a", encoding="utf-8") as f:
        if not text.endswith('\n'):
            f.write('\n')
            f.write(line)
            
    fobj.close()
    f.close()

In [ ]:
def readRecentPlays():
    #data = []
    with open(recent_played_path, 'r', encoding="utf-8") as f:
        data = f.readlines()
        
    f.close()
    return data



def readPlayCount():
    #data = []
    with open(play_count_path, 'r', encoding="utf-8") as f:
        data = f.readlines()

    f.close()
    return data


def writeLinePlayCount(line):
    with open(play_count_path, encoding="utf-8") as fobj:
        text = fobj.read()

    with open(play_count_path, "a", encoding="utf-8") as f:
        if not text.endswith('\n'):
            f.write('\n')
            f.write(line)

    fobj.close()
    f.close()

def updatePlayCount(fileNameString):
    global freshPlay
    #count = line.split()[1]

    b_file = open(play_count_path, "r", encoding="utf-8")
    linesTuple = []

    lines = b_file.readlines()

    for line in lines:
        linesTuple.append((line[:-(2 + len(lines[0].split()[-1]))] ,line.split()[len(line.split())-1]))



    b_file.close()

    c = sorted(linesTuple,key=itemgetter(1))

    #c.reverse()


    new_file = open(play_count_path, "w+", encoding="utf-8")

    for line in c:
        fileName = line[0]
        count = line[1]
        new_file.write(str((fileName + " " + str(count) + "\n")))

    new_file.close()


    fileName = fileNameString


    a_file = open(play_count_path, "r", encoding="utf-8")

    linesStripped = []

    lines = a_file.readlines()

    for line in lines:

        linesStripped.append(line[:-(2 + len(line.split()[-1]))])

    print(len(lines))

    a_file.close()

    tupleList = []

    for line in lines:
        #solution to double space problem
        #find length of integer in final element of splitted list, then use as index
        tupleList.append((line[:-(2 + len(line.split()[-1]))], int(line.split()[-1])))

    freshPlay = sorted(tupleList,key=itemgetter(1))[0][0]

    try:

        new_file = open(play_count_path, "w+", encoding="utf-8")

        if len(lines)<1:
            print("Writing to clear file")
            new_file.write(str((fileName + " " + str(1) + "\n")))

        try:
            linesStripped.index(fileNameString)
            print("Found ", fileNameString, "Updating..")

        except Exception as e:
            print("Not found ", fileNameString, "writing new entry")
            print("New entry to write: ", fileNameString + " 1")
            new_file.write(fileNameString + " 1" + "\n")


        for line in lines:
            if line[:-(2 + len(line.split()[-1]))] != fileNameString:
                #print(line[:-3])
                #print(linetest)
                #print("Writing new line")
                new_file.write(line)
            else:
                print("updating existing line")
                count = int(line.split()[-1])
                fileName = line[:-(2 + len(line.split()[-1]))]

                print(str((fileName, count)))

                new_file.write(str((fileName + " " + str(count + 1) + "\n")))

        new_file.close()


    except Exception as e:
        print("Failed writing or updating printing previous state")
        for line in lines:
            new_file.write(line)



In [ ]:
def addFilesToPlaylist(cPlaylist):
    
    player = vlc.Instance()
    media_player = vlc.MediaListPlayer()
    media_list = player.media_list_new()
    for file in cPlaylist:
        media = player.media_new(file[0])
        media_list.add_media(media)
        
    media_player.set_media_list(media_list)
    return player, media_player

In [ ]:
def sendIndexToRecyclebin( playIndex):
    global player
    global currentlyPlaying
    global cPlaylist
    player.stop()
    time.sleep(1)
    send2trash(currentlyPlaying)
    #send2trash(cPlaylist[playIndex])
    player = playNextRand()
    #player = playNextInc(player)
    return player

In [ ]:
def forceFrontKeys():
    bringWindowToForeground()
    #set_active_window(pywintypes.HANDLE(window[0]))

In [ ]:
# def autoScan(player):
#     global autoSpeed
#     autoSpeed = 10
#     player.set_position(0.1)
#     time.sleep(autoSpeed)
#     if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive()):
#         print("Active breaking")
#         break
    
#     player.set_position(0.25)
#     time.sleep(autoSpeed)
#     if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive()):
#         print("Active breaking")
#         break
    
#     player.set_position(0.5)
#     time.sleep(autoSpeed)
#     if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive()):
#         print("Active breaking")
#         break
    
#     player.set_position(0.75)
#     time.sleep(autoSpeed)
#     if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive()):
#         print("Active breaking")
#         break
    
#     player.set_position(0.95)
#     time.sleep(autoSpeed)
#     if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive()):
#         print("Active breaking")
#         break
    
#     return player
    

In [ ]:
def runAutoplay(player):
    global autoplaying
    global uniqueOnly

    autoplaying = True
    uniqueOnly = False
    player.audio_set_volume(40)
    print("Running autoplay...")
    
    while(player.get_fullscreen() == 0):
        
        if(player.get_fullscreen() == 0):
            player = playNextRand()

            global autoSpeed
            autoSpeed = 10
            player.set_position(0.1)
            time.sleep(autoSpeed)
            try:
                if(pyautogui.getWindowsWithTitle("VLC (Direct3D11 output)")[0].isActive):
                    print("Active breaking")
                    break
                print("Setting position 0.25")
                player.set_position(0.25)
                time.sleep(autoSpeed)
                if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive):
                    print("Active breaking")
                    break
                print("Setting position 0.5")
                player.set_position(0.5)
                time.sleep(autoSpeed)
                if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive):
                    print("Active breaking")
                    break
                print("Setting position 0.75")
                player.set_position(0.75)
                time.sleep(autoSpeed)
                if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive):
                    print("Active breaking")
                    break

                player.set_position(0.95)
                time.sleep(autoSpeed)
                if(pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].isActive):
                    print("Active breaking")
                    break
            except:
                print("Broken on window title")
                player = playNextRand()
                pass
            
            
            
        if(player.get_fullscreen() == 1):
            print("breaking autoplay")

            break
        
        
    print("Done autoplaying")
    return player
        
        
    

In [ ]:
def replay():
    global playIndex
    global player
    global currentlyPlaying
    try:
        pyautogui.getWindowsWithTitle("VLC (Direct3d11 output)")[0].close()

    except Exception as e:
        print(e)

    player.stop()



    if(isTwoVLC()):
        print("Closing double window")
        player.stop()

    #player = vlc.MediaPlayer(cPlaylist[playIndex])
    player = vlc.MediaPlayer()
    media = vlc.Media(currentlyPlaying)
    player.set_media(p_md=media)
    
    #player = vlc.MediaPlayer(currentlyPlaying)

    player.play()

    time.sleep(sleepTime*4)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()

    if(isVLCOpen() is False):
        print("Playback failed on " + str(currentlyPlaying))
        player = vlc.MediaPlayer()
        
        media = vlc.Media(currentlyPlaying)
        player.set_media(p_md=media)
        player.play()
        #player.play(currentlyPlaying)
        #playNextRand()

    time.sleep(sleepTime*4)
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()


    return player

In [ ]:
print(directoryList)

In [ ]:
print(directoryList[0])
cPlaylist = cleanupSortedList(getSortedList(1, directoryList[0]))



with open("cPlaylist", 'wb') as f:
    pickle.dump(cPlaylist, f)

In [ ]:
print(directoryList[0])

In [ ]:
rec = readRecentPlays()
print(len(cPlaylist))
rec2 = []
for i in tqdm(range(0, len(rec))):
    rec2.append(rec[i])

purgedList = []
cplaytemp = []

cplaySingle = []
for i in range(0, len(cPlaylist)):
    cplaySingle.append(cPlaylist[i][0])

if(uniqueOnly):


    cPlaylist = list(set(cplaySingle) - set(rec2))
    #cPlaylist2 = [x for x in cplaySingle if x not in rec2]

#cPlaylist = cplaytemp
print(len(cPlaylist))

In [ ]:
print(uniqueOnly)


In [ ]:
print(len(list(set(rec2).difference(cplaySingle))))

In [ ]:
def writeDuplicate(line):
    with open(dup_list_path, 'a', encoding='utf-8') as f:
        f.write(line+'\n')
        f.close()




In [ ]:
player = vlc.MediaPlayer()
#player = vlc.MediaPlayer(cPlaylist[5][0])
time.sleep(2)


In [ ]:
#player = playNextRand()

In [ ]:
wasFullScreen = False
player = skipFwdLrg(player)
time.sleep(1)
bringWindowToForeground()
player.audio_set_volume(10)

In [ ]:
# def bringWindowToForeground():
#     print("dummy foreground")
#     pass

In [ ]:
global enabled
enabled = True

In [ ]:
SHIFT_STATE = False
# The key combination to check
def get_vk(key):
    """
    Get the virtual key code from a key.
    These are used so case/shift modifications are ignored.
    """
    return key.vk if hasattr(key, 'vk') else key.value.vk
pressed_vks = set()
def is_combination_pressed(combination):
    pressed_vks = set()
    """ Check if a combination is satisfied using the keys pressed in pressed_vks """
    return all([get_vk(key) in pressed_vks for key in combination])

breakKeys = [{keyboard.Key.shift, keyboard.KeyCode(char='Z')}]

recycleKeys = [{ keyboard.KeyCode(char='`')}]



largeFWDkeys = [{keyboard.KeyCode(char='d')}]
largeBWDkeys = [{keyboard.KeyCode(char='a')}]

volIncKeys = [{keyboard.KeyCode(char='e')}]
volDeccKeys = [{keyboard.KeyCode(char='q')}]


fullscreenKeys = [{keyboard.KeyCode(char='f')}]
leftScreenKeys = [{keyboard.KeyCode(char='t')}]


nextRandomKeys = [{keyboard.KeyCode(char='\\')}]

sleepKeys = [{keyboard.Key.ctrl}]

pauseKeys = [{keyboard.Key.space}]

enableKeys = [{keyboard.Key.ctrl_r}]
                                
autoplayKeys = [{keyboard.Key.page_up}]

tinyFWDKeys =[{keyboard.Key.right}]

 
tinyBWDKeys =[{keyboard.Key.left}]

forceFrontKeys = [{keyboard.KeyCode(char='r')}]

# The currently active modifiers
current = set()

def executeforceFrontKeys():
    replay()


def executeTinyFWD():
    try:
        skipFwdTiny()
    except Exception as e:
        print(e)
    
    
def executeTinyBWD():
    try:
        skipBwdTiny()
    except Exception as e:
        print(e)
    
                                
def toggleAutoplay():
    global player
    player = runAutoplay(player)
    #return player
                                
                                  
def toggleEnable():
    global enabled
    if(enabled):
        print("Disabling")  
        enabled = False
    else:
        enabled = True
        print("Enabling")

                                  

def executeLargeFWD():
    global player
    #print ("executeLargeFWD")
    player = skipFwdLrg(player)
   #return player

def executeLargeBWD():
    global player
    #print ("executeLargeBWD")
    skipBwdLrg()    
    #return player
    
def executeNextRand():
    global player
    global wasFullScreen
    wasFullScreen = player.get_fullscreen() == 1
    print('wasFullScreen', wasFullScreen)
    #print("executeNextRand")
    player = playNextRand()
    #return player
    

    
def executePause():
    pause()
    
    

def executeIncVol():
    incVol()
    
    
def executeDecVol():
    decVol()
    

def executeFullscreen():
    toggleFullscreen()
    
def executeLeftScreen():
    print("fixing screen")
    win32gui.EnumWindows(resizeVLCRight, None)
    bringWindowToForeground()
    
    
    
def executeBreak():
    print("Stopping")
    listener.stop()

    
    
def executeRecycle():
    print("Executing recycle")
    global playIndex
    global player
    wasFullScreen = player.get_fullscreen() == 1
    
    try:
        print("execute recycle sending ", )
        #player = sendIndexToRecyclebin(player, playIndex)
        sendIndexToRecyclebin(playIndex)
    except Exception as e:
        print(e)
        
    if(wasFullScreen):
        player.set_fullscreen(True)
    else:
        player.set_fullscreen(False)

    #return player

def executeSleep():
    print("sleeping")
    time.sleep(30)

def on_press(key):

    global SHIFT_STATE
    # if(key == keyboard.Key.down):
    #     executeNextRand()

    if key == keyboard.Key.shift:
        SHIFT_STATE = True
    else:
        try:



            if SHIFT_STATE:
                #print(f'shift + {key}')
                if(key == keyboard.KeyCode(char='Z')):
                    print('shift z terminating')
                    player.stop()
                    KillDrivers.killPythonMPCProc()
                    executeBreak()

                elif(key == keyboard.KeyCode(char='R')):
                    print('shift R replaying')
                    executeforceFrontKeys()

                elif(key == keyboard.Key.space):
                    print('shift space pausing')
                    executePause()


                elif(key == keyboard.KeyCode(char='T')):
                    print('shift t resizing')
                    executeLeftScreen()

                # elif(key == keyboard.KeyCode(char='W')):
                #     print('shift w playnext random')
                #     executeNextRand()


                elif(key == keyboard.KeyCode(char='2')):
                    print('shift 2 playnext random')
                    executeNextRand()


                elif(key == keyboard.KeyCode(char='F')):
                    print('shift f fullscreen')
                    executeFullscreen()

                elif(key == keyboard.KeyCode(char='E')):
                    print('shift e vol up')
                    executeIncVol()

                elif(key == keyboard.KeyCode(char='Q')):
                    print('shift q vol down')
                    executeDecVol()



                # elif(key == keyboard.KeyCode(char='D')):
                #     print('shift d forward')
                #     executeLargeFWD()
                # 
                # elif(key == keyboard.KeyCode(char='A')):
                #     print('shift a back')
                #     executeLargeBWD()


                else:
                    pass
                    #print(key)



            else:
                #print(key)
                pass
        except Exception as e:
            print(e)

    """ When a key is pressed """
    vk = get_vk(key)  # Get the key's vk
    pressed_vks.add(vk)  # Add it to the set of currently pressed keys

    if any([key in COMBO for COMBO in enableKeys]):
            current.add(key)               
            if any(all(k in current for k in COMBO) for COMBO in enableKeys):
                    toggleEnable()
    
    if(enabled):
        if any([key in COMBO for COMBO in largeFWDkeys]):
            current.add(key)
            if any(all(k in current for k in COMBO) for COMBO in largeFWDkeys):
                executeLargeFWD()

        if any([key in COMBO for COMBO in largeBWDkeys]):
            current.add(key)
            if any(all(k in current for k in COMBO) for COMBO in largeBWDkeys):
                executeLargeBWD()





        if any([key in COMBO for COMBO in breakKeys]):
            for combination in breakKeys:  # Loop though each combination
                if is_combination_pressed(combination):  # And check if all keys are pressed
                    executeBreak()
                    #execute()  # If they are all pressed, call your function
                    break  # Don't allow execute to be called more than once per key press



        if any([key in COMBO for COMBO in nextRandomKeys]):
            current.add(key)
            if any(all(k in current for k in COMBO) for COMBO in nextRandomKeys):
                    executeNextRand()

        if any([key in COMBO for COMBO in recycleKeys]):
            current.add(key)               
            if any(all(k in current for k in COMBO) for COMBO in recycleKeys):
                    executeRecycle()

        if any([key in COMBO for COMBO in sleepKeys]):
            current.add(key)               
            if any(all(k in current for k in COMBO) for COMBO in sleepKeys):
                    executeSleep()                

        # if any([key in COMBO for COMBO in pauseKeys]):
        #         current.add(key)
        #         if any(all(k in current for k in COMBO) for COMBO in pauseKeys):
        #                 pause()



        if any([key in COMBO for COMBO in autoplayKeys]):
                current.add(key)               
                if any(all(k in current for k in COMBO) for COMBO in autoplayKeys):
                        toggleAutoplay()

        if any([key in COMBO for COMBO in tinyFWDKeys]):
                current.add(key)               
                if any(all(k in current for k in COMBO) for COMBO in tinyFWDKeys):
                        executeTinyFWD()

        if any([key in COMBO for COMBO in tinyBWDKeys]):
                current.add(key)               
                if any(all(k in current for k in COMBO) for COMBO in tinyBWDKeys):
                        executeTinyBWD()


                        
         
        # if any([key in COMBO for COMBO in forceFrontKeys]):
        #         current.add(key)
        #         if any(all(k in current for k in COMBO) for COMBO in forceFrontKeys):
        #                 executeforceFrontKeys()
        
                        
                    
        
        
# def on_release(key):
#     #if any([key in COMBO for COMBO in breakKeys]):
#         pass
#      #   current.remove(key)

def on_release(key):
    #print('{0} release'.format(
     #   key))
    global SHIFT_STATE
    if key == keyboard.Key.esc:
        # Stop listener
        return False
    elif key == keyboard.Key.shift:
        SHIFT_STATE = False
    if key == Key.esc:
        # Stop listener
        return False


def on_click(x, Wy, button, pressed):
    # print('{0} at {1}'.format(
    #     'Pressed' if pressed else 'Released',
    #     (x, y)))
    #print(button)

    if(button == Button.x1 and pressed):


        executeLargeBWD()

    elif(button == Button.x2 and pressed):
        
        executeNextRand()
        return True


    elif(button == Button.middle and pressed):
        executeLargeFWD()

    return True




    # if not pressed:
    #     # Stop listener
    #     return False
with keyboard.Listener(on_press=on_press, on_release=on_release)  as listener:
    with mouseListener(on_click=on_click)  as listener:

        #listener.start()
        listener.join()





# with keyboard.Listener(on_press=on_press, on_release=on_release)  as listener1:
#     listener1.join()



In [ ]:
playNextRand()

In [ ]:
#pip install pygetwindow

In [ ]:
player.play()

In [ ]:
player.set_fullscreen(True)

In [ ]:
player = vlc.MediaPlayer()

In [ ]:
media = vlc.Media()
player.set_media(p_md=media)

In [ ]:
player.stop()